# Database Copy and Backup

A quick example of using [sqlite3 backup][back] methods to copy and backup databases.

These work because you can specify an `sqlite3.Connection` object to instantiate JSONLiteDB

[back]:https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.backup

In [1]:
import os, sys

# This is just to make sure running the devel version.
# NOT NEEDED once JSONLiteDB is installed
p = os.path.abspath("../")
if p not in sys.path:
    sys.path.insert(0, p)

import jsonlitedb
from jsonlitedb import JSONLiteDB, Q

In [2]:
import sqlite3
import subprocess

In [3]:
db0 = JSONLiteDB(":memory:")
print(db0)

JSONLiteDB(':memory:')


In [4]:
db0.insert(
    {"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"},
    {"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"},
    {"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"},
    {"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"},
    {"first": "George", "last": "Martin", "born": 1926, "role": "producer"},
)

Copy it by createing a new database connection

In [5]:
# TMP sqlite3 connection
_db1 = sqlite3.connect(":memory:")

# Use the underlying sqlite3.Connection object of db0 to backup
db0.db.backup(_db1)

# create a new database
db1 = JSONLiteDB(_db1)
print(db1)

JSONLiteDB('*existing connection*')


In [6]:
list(db1)

[{'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'},
 {'first': 'Paul', 'last': 'McCartney', 'born': 1942, 'role': 'bass'},
 {'first': 'George', 'last': 'Harrison', 'born': 1943, 'role': 'guitar'},
 {'first': 'Ringo', 'last': 'Starr', 'born': 1940, 'role': 'drums'},
 {'first': 'George', 'last': 'Martin', 'born': 1926, 'role': 'producer'}]

Demonstrate that they are not the same underlying database

In [7]:
db1.delete(last="Martin")
print(f"{db0.count(last='Martin') = }")
print(f"{db1.count(last='Martin') = }")

db0.count(last='Martin') = 1
db1.count(last='Martin') = 0


The same approach can be used to save an in-memory database too

In [8]:
try:  # Just to make sure it gets cleared
    dbfile = sqlite3.connect("mydb.db")
    db0.db.backup(dbfile)
    dbfile.close()

    print(f"{os.path.exists('mydb.db') = }")

    db2 = JSONLiteDB("mydb.db")
    print(f"{len(db2) = }")

    print(subprocess.check_output(["sqlite3", "mydb.db", ".dump"]).decode())
finally:
    pass  # os.unlink('mydb.db') # This will also fail if mydb.db didn't exist

os.path.exists('mydb.db') = True
len(db2) = 5
PRAGMA foreign_keys=OFF;
BEGIN TRANSACTION;
CREATE TABLE items(
    rowid INTEGER PRIMARY KEY AUTOINCREMENT,
    data TEXT
);
INSERT INTO items VALUES(1,'{"first":"John","last":"Lennon","born":1940,"role":"guitar"}');
INSERT INTO items VALUES(2,'{"first":"Paul","last":"McCartney","born":1942,"role":"bass"}');
INSERT INTO items VALUES(3,'{"first":"George","last":"Harrison","born":1943,"role":"guitar"}');
INSERT INTO items VALUES(4,'{"first":"Ringo","last":"Starr","born":1940,"role":"drums"}');
INSERT INTO items VALUES(5,'{"first":"George","last":"Martin","born":1926,"role":"producer"}');
CREATE TABLE items_kv(
    key TEXT PRIMARY KEY,
    val TEXT
);
INSERT INTO items_kv VALUES('created','2026-02-08T15:19:12.894516-07:00');
INSERT INTO items_kv VALUES('version','JSONLiteDB-0.1.12');
DELETE FROM sqlite_sequence;
INSERT INTO sqlite_sequence VALUES('items',5);
COMMIT;

